## Importando as bibliotecas

In [57]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
pd.set_option("display.max_rows", 200)
pd.set_option('display.max_columns', 500)

### Leitura do arquivo de treino do dataset

In [58]:
train = pd.read_csv('train.csv')

### Verificando a correlação das variáveis com a target

In [59]:
train.corr().round(4)['IN_TREINEIRO']

Unnamed: 0                     0.0067
NU_ANO                            NaN
CO_MUNICIPIO_RESIDENCIA       -0.0353
CO_UF_RESIDENCIA              -0.0349
NU_IDADE                      -0.2951
TP_ESTADO_CIVIL               -0.1179
TP_COR_RACA                    0.0097
TP_NACIONALIDADE              -0.0152
CO_MUNICIPIO_NASCIMENTO       -0.0182
CO_UF_NASCIMENTO              -0.0180
TP_ST_CONCLUSAO                0.5340
TP_ANO_CONCLUIU               -0.2577
TP_ESCOLA                     -0.2446
TP_ENSINO                         NaN
IN_TREINEIRO                   1.0000
CO_ESCOLA                         NaN
CO_MUNICIPIO_ESC                  NaN
CO_UF_ESC                         NaN
TP_DEPENDENCIA_ADM_ESC            NaN
TP_LOCALIZACAO_ESC                NaN
TP_SIT_FUNC_ESC                   NaN
IN_BAIXA_VISAO                -0.0136
IN_CEGUEIRA                       NaN
IN_SURDEZ                     -0.0074
IN_DEFICIENCIA_AUDITIVA        0.0009
IN_SURDO_CEGUEIRA                 NaN
IN_DEFICIENC

### Selecionando as variáveis para o modelo. A de menor correlação foi  NU_NOTA_COMP1 = 0.299

In [60]:
colunas = ['TP_ST_CONCLUSAO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT',
           'IN_TREINEIRO']

In [61]:
# dataframe somente com as variáveis escolhidas
train = train[colunas]

In [62]:
# verificando a correlação no novo dataframe
train.corr().round(4)

,TP_ST_CONCLUSAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,IN_TREINEIRO
TP_ST_CONCLUSAO,1.0000,0.0548,0.0548,0.0533,0.0533,0.5340
TP_PRESENCA_CN,0.0548,1.0000,1.0000,0.9434,0.9434,0.0947
TP_PRESENCA_CH,0.0548,1.0000,1.0000,0.9434,0.9434,0.0947
TP_PRESENCA_LC,0.0533,0.9434,0.9434,1.0000,1.0000,0.0925
TP_PRESENCA_MT,0.0533,0.9434,0.9434,1.0000,1.0000,0.0925
IN_TREINEIRO,0.5340,0.0947,0.0947,0.0925,0.0925,1.0000


#### Verificando mais informações do dataframe

In [63]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   TP_ST_CONCLUSAO  13730 non-null  int64
 1   TP_PRESENCA_CN   13730 non-null  int64
 2   TP_PRESENCA_CH   13730 non-null  int64
 3   TP_PRESENCA_LC   13730 non-null  int64
 4   TP_PRESENCA_MT   13730 non-null  int64
 5   IN_TREINEIRO     13730 non-null  int64
dtypes: int64(6)
memory usage: 643.7 KB


### Verificando estatísticas descritivas das variáveis

In [64]:
train.describe().round(2)

,TP_ST_CONCLUSAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,IN_TREINEIRO
count,13730.00,13730.00,13730.00,13730.00,13730.00,13730.00
mean,1.78,0.75,0.75,0.74,0.74,0.13
std,0.88,0.43,0.43,0.44,0.44,0.34
min,1.00,0.00,0.00,0.00,0.00,0.00
25%,1.00,1.00,1.00,0.00,0.00,0.00
50%,2.00,1.00,1.00,1.00,1.00,0.00
75%,2.00,1.00,1.00,1.00,1.00,0.00
max,4.00,2.00,2.00,2.00,2.00,1.00


#### Verificando a quantidade de nulos nas variáveis

In [65]:
train.isnull().sum()

TP_ST_CONCLUSAO    0
TP_PRESENCA_CN     0
TP_PRESENCA_CH     0
TP_PRESENCA_LC     0
TP_PRESENCA_MT     0
IN_TREINEIRO       0
dtype: int64

#### Selecionando a target do modelo

In [67]:
y_train = train.IN_TREINEIRO
y_train

0        0
1        0
2        0
3        0
4        0
        ..
13725    0
13726    0
13727    1
13728    0
13729    0
Name: IN_TREINEIRO, Length: 13730, dtype: int64

#### Selecionando as variáveis independente

In [68]:
X_train = train
X_train.drop(columns=['IN_TREINEIRO'], inplace=True)

In [69]:
# Verificando quais são as variáveis independentes
X_train.columns

Index(['TP_ST_CONCLUSAO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT'],
      dtype='object')

In [70]:
colunas

['TP_ST_CONCLUSAO',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'IN_TREINEIRO']

### Criando lista com as colunas que foram usadas para a parte do treino, sabendo que tem que ser igual para o teste

In [71]:
# criando uma cópia da lista
colunas_test = colunas.copy()

# removendo a última variável da lista, neste caso é IN_TREINEIRO, que no caso é a target e não existe no teste
colunas_test.pop()

# adicionando o NU_INSCRICAO
colunas_test.append('NU_INSCRICAO')

### Leitura do arquivo de teste do dataset

In [72]:
test = pd.read_csv('test.csv', usecols=colunas_test)

#### Salvando a lista de identificação dos participantes do enem para montar o arquivo ao final do modelo

In [73]:
num_inscricao = test['NU_INSCRICAO']

#### Verificando a quantidade de nulos nas variáveis

In [74]:
test.isnull().sum()

NU_INSCRICAO       0
TP_ST_CONCLUSAO    0
TP_PRESENCA_CN     0
TP_PRESENCA_CH     0
TP_PRESENCA_LC     0
TP_PRESENCA_MT     0
dtype: int64

#### Criando coluna target no dataframe de teste, valores nulos pois o modelo irá fazer a predição

In [77]:
test['IN_TREINEIRO'] = None

#### Selecionando a target do modelo

In [78]:
y_test = test['IN_TREINEIRO']
y_test

0       None
1       None
2       None
3       None
4       None
        ... 
4565    None
4566    None
4567    None
4568    None
4569    None
Name: IN_TREINEIRO, Length: 4570, dtype: object

#### Selecionando as variáveis independente

In [79]:
# excluindo NU_INSCRICAO, porque não faz sentido para o modelo
test.drop(columns=['NU_INSCRICAO'], inplace=True)
X_test = test

In [80]:
X_test.drop(columns=['IN_TREINEIRO'], inplace=True)
X_test

,TP_ST_CONCLUSAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT
0,1,1,1,0,0
1,1,0,0,0,0
2,3,1,1,0,0
3,2,1,1,1,1
4,1,1,1,1,1
...,...,...,...,...,...
4565,2,1,1,1,1
4566,1,1,1,1,1
4567,1,1,1,1,1
4568,2,1,1,1,1


## Instanciando modelo RandomForestClassifier

In [81]:
modelo = RandomForestClassifier(n_estimators=2000, n_jobs=-1, max_depth=100, min_samples_leaf=4,
                             min_samples_split=10, random_state=0)

### Treinando modelo

In [83]:
%%time
modelo.fit(X_train, y_train)

CPU times: user 5.74 s, sys: 597 ms, total: 6.34 s
Wall time: 3.51 s


RandomForestClassifier(max_depth=100, min_samples_leaf=4, min_samples_split=10,
                       n_estimators=2000, n_jobs=-1, random_state=0)

### Verificando a acurácia

#### Na classificação de vários rótulos, essa é a precisão do subconjunto, que é uma métrica severa, pois você precisa para cada amostra que cada conjunto de rótulos seja previsto corretamente.

In [84]:
print('acurácia = {}'.format(modelo.score(X_train, y_train).round(2)))

acurácia = 0.96


### Classificando a variável target para o teste

In [90]:
y_predict = modelo.predict(X_test)
y_predict

array([0, 0, 1, ..., 0, 0, 0])

In [98]:
# Criando uma Serie para as classificações
y_predict = pd.Series(y_predict)
y_predict

0       0
1       0
2       1
3       0
4       0
       ..
4565    0
4566    0
4567    0
4568    0
4569    0
Length: 4570, dtype: int64

In [94]:
# Criando dataframe com NUM_INSCRICAO e IN_TREINEIRO (classificado)
answer = pd.concat([num_inscricao, y_predict], axis=1)
# Renomeando coluna
answer.rename(columns={0:'IN_TREINEIRO'}, inplace=True)

In [95]:
answer

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0
...,...,...
4565,361b7fcd8867119550fe2af5aa729ffad89a7cf5,0
4566,d8a0e4c9e29494cc9bba2422bd79333931475ee1,0
4567,3f1c3388244df8d6521e983a809292d9f3bca643,0
4568,1778e9c4cef591beb6b986d191d15ed05de816b0,0


## Criando arquivo para submissão do desafio

In [97]:
answer.to_csv('answer.csv',columns=['NU_INSCRICAO', 'IN_TREINEIRO'], sep=',', index=False)